In [1]:
import pyclifford as pc
import numpy as np

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


## Clifford Gates

`CliffordGate` Represents a Clifford unitary gate.
- **Parameters**:
    -  qubits: int - the qubits that this gate acts on.
- **Attribute**:
    - qubits: a tuple containing the number of qubits that it acts on 
    - generator: Pauli - if the clifford gate is a rotation generated by a single 
        Pauli generator (which is generally not the case), then this records 
        its generator. It is more efficient to implement Clifford rotation than 
        generic Clifford transform. `Default: None`
    - forward_map / backward_map: CliffordMap - a generic Clifford gate will be 
        described by the Clifford map, which is a table specifying how each 
        single Pauli operator gets mapped to. (forward and backward maps must 
        be inverse to each other). `Default: None`
    - unitary: bool - indicates that gate is unitary.
- **Methods**:
    - `CliffordGate.reset()`: reset the Clifford gate to null
    - `CliffordGate.set_generator(Pauli)`
    - `CliffordGate.set_forward_map(CliffordMap)`
    - `CliffordGate.set_backward_map(CliffordMap)` **TODO: we need to add a handle to prevent user set conflict Clifford maps**
    - `CliffordGate.copy()`: return a copy of the Clifford gate
    - `CliffordGate.independent_from(Other_gate)`: if there is no overlap between acting qubits, then they are independent.
    - `CliffordGate.compile()`: build (forward&backward) Clifford map representation. If generator is given, then it will be converted to forward/backward map; if forward map is given, the backward map will be calculated; if backward map is given, the forward map will be calculated
    - `CliffordGate.forward(obj)`: forward transformation of the object. `obj` can be Pauli, PauliList, PauliPolynomial, StabilizerState.

CliffordGate is the low-level API(class) for Clifford gates. One can create a **null** Clifford gate by sepecifying what are the qubits this gate will act on:

Example: `circuit.CliffordGate(1,2)` will create a null gate acting on qubit 1, and 2. Null gate does not contain any `forward_map` or `backward_map`

In [3]:
gate = pc.CliffordGate(1,2)

In [4]:
print("gate acts on: {}, and there are {} qubits the gates will act on.".format(
gate.qubits, gate.n
))

gate acts on: (1, 2), and there are 2 qubits the gates will act on.


In [6]:
print("since this is a null gate, the only information is the gate acts on 1,2 qubits. The forward map is None: ", gate.forward_map)

since this is a null gate, the only information is the gate acts on 1,2 qubits. The forward map is None:  None


Now if we call the forward or backward method, it will randomly generate a Clifford gate.

In [7]:
state = pc.zero_state(3)
gate.forward(state)

(StabilizerState(
    +ZII
    -IXY
    +IIY),
 0.0)

In [8]:
print("now the gate is randomly assigned a forward map: ", gate.forward_map)

now the gate is randomly assigned a forward map:  CliffordMap(
  X0-> +YY
  Z0-> -XY
  X1-> +ZX
  Z1-> +IY)


And if we want to generate a new random Clifford gate, we need to `reset()` it. Luckily, we will see that circuit object also supports the `reset()` method, so we can just call `circuit.reset()` to reset all gates in the circuit.

In [9]:
state = pc.zero_state(3)
gate.forward(state)

(StabilizerState(
    +ZII
    -IXY
    +IIY),
 0.0)

if we want a new random Clifford gate. We do the following:

In [11]:
gate.reset()
state = pc.zero_state(3)
gate.forward(state)

(StabilizerState(
    +ZII
    +IYI
    +IIX),
 0.0)

There are three ways to equip the null gate with Clifford map:
1. One can use `gate.set_generator(Pauli)`. This will create a $\pi/2$ rotation generated by the pauli string.
2. One can use `gate.set_forward_map(CliffordMap)`
3. One can use `gate.set_backward_map(CliffordMap)`

In [12]:
gate.set_generator(pc.pauli("XX"))

In [13]:
gate.set_forward_map(pc.random_clifford_map(2))

In [14]:
gate.set_backward_map(pc.random_clifford_map(2))

`gate.compile()` will construct both `forward_clifford_map` and `backward_clifford_map`. Compilation is not necessary!

`gate.copy()` will return a copy of the `CliffordGate`

One can apply Clifford Gate $U$ or $U^{\dagger}$ to: Pauli strings, Stabilizer States by `gate.forward(obj)` and `gate.backward(obj)`

In [49]:
psi = pc.ghz_state(5)
gate = pc.CliffordGate(1,2)
gate.set_generator(pc.pauli("XX"))
gate.forward(psi)

StabilizerState(
   -ZYXII
   +IZZII
   -IXYZI
   +IIIZZ
   +XXXXX)

We see the wavefunction $|\psi\rangle$ has been changed **in-place**. And the gate only changes qubit 1&2.

If two Clifford gate acting on different qubits, then they are independent. We can check dependency by `gate.independent_from(other_gate)`

In [15]:
gate1 = pc.CliffordGate(0,1)
gate2 = pc.CliffordGate(2,3)
print(gate1.independent_from(gate2))

True


There are many predefined Clifford gates. For example, H, S, X, Y, Z, CNOT. They are simply defined using the forward_map (`CliffordMap`). Here is one example:
```python
def H(*qubits):
    if len(qubits)!=1:
        raise ValueError("Hadmand gate only acts on a single qubit.")
    gate = CliffordGate(*qubits)
    f_map = CliffordMap(gs = np.array([[0,1],[1,0]]),ps = np.array([0,0]))
    gate.set_forward_map(f_map)
    return gate
```

Here is a Hadmand gate example which acting on the 4th qubit:

In [16]:
pc.H(4)

[4]

There are also predefined all 24 single qubit Clifford gates. One can call `C(num, *qubits)` to generate the `num-th` Clifford gate, where `num` is the index of the gate and it is from 0-23.

In [52]:
pc.C(15,0)

[0]

Here is a CNOT gate:

In [53]:
pc.CNOT(0,1)

[0,1]

## Measurement 

Represents a computational basis measurement.
    
**Parameters**:
- qubits: int - the qubits to be measured

**Data**:
- out: int (L) - array of measurement outcomes on corresponding qubits
         None before measurement, populated after forward measurement.
- unitary: bool - indicates that measurement is not unitary.'''

In [30]:
measure = pc.Measurement(0)
state = pc.ghz_state(5)
measure.forward(state)

(StabilizerState(
    +ZZIII
    +IZZII
    +IIZZI
    +IIIZZ
    -ZIIII),
 -1.0)

In [31]:
print("state after the measurement: ", state)

state after the measurement:  StabilizerState(
   +ZZIII
   +IZZII
   +IIZZI
   +IIIZZ
   -ZIIII)


In [32]:
print("the measurement outcome is: ", measure.out)

the measurement outcome is:  [1]


## Layers

Representes a layer of Clifford gate or measurement operations.

**Parameters**:

- *ops: Operation - the operations (gates or measurements) in this layer

**Attributes**:

- prev_layer: Layer - the previous layer
- next_layer: Layer - the next layer
- forward_map: CliffordMap - the forward map of the layer (if unitary)
- backward_map: CliffordMap - the backward map of the layer (if unitary)
- unitary: bool - whether the layer is unitary

`Layer` can take `CliffordGates` and/or `Measurement` by `layer.append(objs)`:

In [33]:
circlayer = pc.Layer()

In [34]:
circlayer.append(pc.CliffordGate(0,1))
circlayer.append(pc.CliffordGate(1,3))

|[0,1][1,3]|

In [35]:
print(circlayer)

|[0,1][1,3]|


In [36]:
psi=pc.ghz_state(4)
print(psi)

StabilizerState(
   +ZZII
   +IZZI
   +IIZZ
   +XXXX)


In [37]:
circlayer.forward(psi)

(StabilizerState(
    -ZXIX
    -XIZY
    -IYZY
    +ZZXY),
 0.0)

And one can also easily add all the gates/measurements in the construction step:

In [31]:
circlayer = pc.Layer(pc.CliffordGate(0,1), pc.CliffordGate(1,3))
state = pc.ghz_state(4)
print("state before the layer: ", state)
state,log2prob = circlayer.forward(state)

state before the layer:  StabilizerState(
   +ZZII
   +IZZI
   +IIZZ
   +XXXX)


In [28]:
print("state after the layer: ", state)

state after the layer:  StabilizerState(
   +IXIY
   +YIZI
   -IXZI
   -XZXX)


<div class="alert alert-block alert-success">
If the forward map and backward map for a gate is Null, then it will be assigned a <b> different </b>random clifford map each time when use calls forward() or backward()
</div>

## Circuit

Represents a quantum process of Clifford circuit with gates and measurements.
**Parameters**:

- *ops: Operation - the operations (gates or measurements) in this circuit

**Attributes**:

- first_layer: Layer - the first layer of the circuit
- last_layer: Layer - the last layer of the circuit
- forward_map: CliffordMap - the forward map of the circuit (if unitary)
- backward_map: CliffordMap - the backward map of the circuit (if unitary)
- unitary: bool - whether the circuit is unitary

If the circuit contains null gates, a forward map will be randomly generated and assigned to those
gates the first time when circuit forward() or backward() is called. And if one wants to generate a new random circuit, one can call `circuit.reset()` to reset all gates in the circuit.

Let's create a circuit with 4 qubits and generate GHZ state:

In [39]:
circ = pc.Circuit(pc.H(0),pc.CNOT(0,1),pc.CNOT(1,2),pc.CNOT(2,3))
print(circ)

Circuit(
  |[2,3]|
  |[1,2]|
  |[0,1]|
  |[0]|)


Now the circuit is ready, and let's acting it to zero state to generate GHZ state.

In [40]:
N=4
state = pc.zero_state(N)
state = circ.forward(state)
print("state after circuit: ", state)

state after circuit:  (StabilizerState(
   +XXXX
   +ZZII
   +IZZI
   +IIZZ), 0.0)


We have successfully generated a GHZ state!

Now let's do something fun. We create a mid-circuit measurement in the GHZ circuit:

In [41]:
N = 4
circ = pc.Circuit()
circ.append(pc.H(0))
circ.append(pc.CNOT(0,1))
circ.measure(1)
circ.append(pc.H(1))
circ.append(pc.CNOT(1,2))
circ.append(pc.CNOT(2,3))
print(circ)

Circuit(
  |[2,3]|
  |[1,2]|
  |[1]|
  |<1>|
  |[0,1]|
  |[0]|)


In [42]:
state = pc.zero_state(N)
state = circ.forward(state)
print("state after circuit: ", state)

state after circuit:  (StabilizerState(
   -IXXX
   +ZXXX
   +IZZI
   +IIZZ), -1.0)


In [45]:
print("The mid-circuit measurement is also saved: ",circ.out)

The mid-circuit measurement is also saved:  [1]


## Examples

### Create random MERA state

In [48]:
circ = pc.Circuit()
circ.append(pc.CliffordGate(0,4))
circ.append(pc.CliffordGate(2,4))
circ.append(pc.CliffordGate(0,2))
circ.append(pc.CliffordGate(4,6))
circ.append(pc.CliffordGate(1,2))
circ.append(pc.CliffordGate(3,4))
circ.append(pc.CliffordGate(5,6))
circ.append(pc.CliffordGate(0,1))
circ.append(pc.CliffordGate(2,3))
circ.append(pc.CliffordGate(4,5))
circ.append(pc.CliffordGate(6,7))
state = pc.zero_state(8)
circ.forward(state)

(StabilizerState(
    -IZIXYZXY
    +ZXXYIIII
    +YIZYZZXY
    +IIIYYXII
    -YZIIIIII
    -IIIIIXYI
    -IIYXIYZY
    +IIIIIIYZ),
 0.0)

In [49]:
state

StabilizerState(
   -IZIXYZXY
   +ZXXYIIII
   +YIZYZZXY
   +IIIYYXII
   -YZIIIIII
   -IIIIIXYI
   -IIYXIYZY
   +IIIIIIYZ)